# Combine trait-mapping data with peptide predictions and summarize

This notebook creates a summary TSV file that records incidence of peptide predictions within each orthogroup that is associated with the itch suppression trait.
This information should help us select orthogroups that have:
1. stronger peptide predictions (peptide predictions many proteins in the orthogroup)
2. strong support for itch suppression. The strongest signal would be protein families from tick + non-tick that suppress itch. To a certain extent the coefficients should help identify these.

Some thoughts from Austin included below.
From Austin in slack (edited for clarity):
> As for mixed orthogroups. Yes, I would absolutely expect these orthogroups to possess proteins/peptides present in both itch suppressing and non-itch suppressing species.
> * These tests are not looking for pure presence/absence patterns (i.e. completely absent in non-suppressing and present in suppressing species) - they are looking for gross patterns.
> * At the level of profile clusters, I took the mean counts of different event types, e.g. speciation events in the gene family (any bifurcation in the gene family tree) within itch suppressing species, and within non-itch suppressing species.
> * I then carried out logistic regressions asking whether the mean event counts predicted itch suppression (binary, 0/1).
>    * So long as the counts significantly differed between itch suppressing and non-itch suppressing species (and the coefficients are sufficiently large so as to be retained, depending on how you filtered), they will be included in your set of “itch suppression-associated” clusters
> * Within profile cluster, I then conducted logistic regressions for each gene family individual, this time regressing the event counts for each species, against the response - the binary itch suppression trait.
>    * One key bit is that this time i used a phylogenetic logistic regression that accounts for the evolutionary non-independence.
>    * That will ultimately mean that in a hypothetical scenario where all ticks have one or more gene copy of something, but no other species do (including other non-tick itch suppressing species), this is unlikely (or less likely) to lead to a significant association. This is because although we have a bunch of the ticks in the dataset, they are effectively evolutionary pseudoreplicates, as that gene only is associated with a single evolutionary “origin” or incidence of itch suppression.
>    * The association between that gene and itch suppression is confounded with the association between that gene and… ticks. And all other traits that are unique to them, independent of itch suppression. 
> * This is all to say… These are just statistical associations between gene family event counts in profile clusters or gene families within them and itch suppression.
>    * Because they are associations between these different event types and not just gene presence/absence, they will include cases where the gene is typically absent in one group and present in the other, as well as cases where it’s present in both, but the “magnitude” or frequency of event counts differs.
>
> As final thought: I think one thing that would be particularly useful to consider in your filtering is to not only consider those gene families where peptides are present in multiple ticks, but multiple ticks as well as other non-tick itch suppressing species. To be fair, the statistical tests should already be “prioritizing” these.

In [1]:
library(tidyverse)
library(UpSetR)

── Attaching core tidyverse packages ────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
setwd("..")

In [3]:
# adjust plot size rendered inline
options(repr.plot.width = 6, repr.plot.height = 4, repr.plot.res = 300)

## Define some descriptive variables associated with trait mapping

In [4]:
evidence_of_itch_suppression_species <- c("Sarcoptes scabiei",
                                          "Psoroptes ovis",
                                          "Amblyomma americanum",
                                          "Amblyomma sculptum",
                                          "Dermacentor andersoni",
                                          "Dermacentor silvarum",
                                          "Dermacentor variabilis",
                                          "Haemaphysalis longicornis",
                                          "Hyalomma asiaticum",
                                          "Ixodes persulcatus",
                                          "Ixodes ricinus",
                                          "Ixodes scapularis",
                                          "Rhipicephalus microplus",
                                          "Rhipicephalus sanguineus")

In [5]:
ticks <- c("Amblyomma americanum",
           "Amblyomma sculptum",
           "Dermacentor andersoni",
           "Dermacentor silvarum",
           "Dermacentor variabilis",
           "Haemaphysalis longicornis",
           "Hyalomma asiaticum",
           "Ixodes persulcatus",
           "Ixodes ricinus",
           "Ixodes scapularis",
           "Rhipicephalus microplus",
           "Rhipicephalus sanguineus")

## Read in initial data

Note that for the peptigate results, we remove propeptide predictions -- DeepPeptide predict peptides and propeptides.
The tool uses the [UniProt definition of propeptide](https://www.uniprot.org/help/propep).
> A **Propeptide** is a part of a protein that is cleaved during maturation or activation of the protein. It is generally understood not to have an independent function.
A **Peptide** is proteolytically cleaved and has a well-defined biological activity.

Given this, we remove propeptide predictions as DeepPeptide would predict that these are not biologically active after being cleaved from their precursor protein.

In [6]:
# tot peptigate predictions & remove propeptide predictions
peptigate_predictions <- read_tsv("outputs/ToT_20240626/predictions/peptide_predictions.tsv", show_col_types = F) %>%
  rename_with(.cols = everything(), function(x){paste0("peptigate_", x)}) %>%
  mutate(peptigate_peptide_class = ifelse(is.na(peptigate_peptide_class), "sORF", peptigate_peptide_class)) %>%
  filter(peptigate_peptide_class != "Propeptide") %>%
  mutate(traitmapping_locus_tag = gsub("_start.*", "", peptigate_peptide_id))

In [7]:
tail(peptigate_predictions)

peptigate_peptide_id,peptigate_start,peptigate_end,peptigate_peptide_type,peptigate_peptide_class,peptigate_prediction_tool,peptigate_nlpprecursor_class_score,peptigate_nlpprecursor_cleavage_score,peptigate_X3.x,peptigate_protein_sequence,peptigate_X3.y,traitmapping_locus_tag
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>,<lgl>,<chr>
Rhipicephalus-sanguineus_XP-037517499.1,NA,NA,sORF,sORF,less_than_100aa,NA,NA,NA,MKMIIFTITLLIISDYVWPCDATGSTICSRAFMVIRKSFGCPFGSNECSKYCKEKKAQKGGYCTGRFKENCVCFRN,NA,Rhipicephalus-sanguineus_XP-037517499.1
Rhipicephalus-sanguineus_XP-037521633.1,NA,NA,sORF,sORF,less_than_100aa,NA,NA,NA,MKLLTIFLIFGLVSGILATASLNENKSEVAHLRVRRWGFGCPFNQRACHRHCRSIRRRAGYCGGRFKLTCTCVRR,NA,Rhipicephalus-sanguineus_XP-037521633.1
Ricinoides-atewa_comp77536-c0-seq1.p1,NA,NA,sORF,sORF,less_than_100aa,NA,NA,NA,RAWENLAAEYNSRPGVHRRTGTQLRKLWVNLKTRNKKEGKEMALDRPTIVREKPERPTSISPTTMKVIQLVPSLMRSFEGGDSDLEEDLQSTVHIMLWPT,NA,Ricinoides-atewa_comp77536-c0-seq1.p1
Ricinoides-atewa_comp81574-c0-seq1.p1,NA,NA,sORF,sORF,less_than_100aa,NA,NA,NA,MIRERAKMIFNTEIDEESKRKETFVASSGWLQKFMERNSLSLRRKTTVAQHDPDKLIDKVVSFVMWISQLRMTKSIPECNIIGMDETSVWFDMPGETTV,NA,Ricinoides-atewa_comp81574-c0-seq1.p1
Tyrophagus-putrescentiae_KAH9391113.1,NA,NA,sORF,sORF,less_than_100aa,NA,NA,NA,MALLNVNFVWSPVTVFVAAGKTFGCPLTSSCRKHCQMNKFKGGQCEGNLRLLMNPICTNIAPN,NA,Tyrophagus-putrescentiae_KAH9391113.1
Tyrophagus-putrescentiae_KAH9391114.1,NA,NA,sORF,sORF,less_than_100aa,NA,NA,NA,MKLFGQVFSLLLLLTIGLVMMSTSVSADYGCPITSKCKQHCLENKFKSGSCEGTLKLTCHCVG,NA,Tyrophagus-putrescentiae_KAH9391114.1


In [8]:
# transcriptome shotgun assembly (TSA) salivary gland (sg) transcriptome peptide prediction BLAST hits
tsa_sg_blastp <- read_tsv("outputs/analysis/compare_tsa_sg/tsa_sg_peptides_blastp_matches.tsv", show_col_types = FALSE) %>%
  # select only one blast hit for each query peptide
  group_by(qseqid) %>%
  slice_max(bitscore) %>%
  slice_min(evalue) %>%
  slice_head(n = 1) %>%
  ungroup() %>%
  filter(qseqid %in% peptigate_predictions$peptigate_peptide_id) %>% # filter out hits to propeptides
  rename_with(.cols = everything(), function(x){paste0("sgpeptide_blast_", x)})

In [9]:
# metadata from trait mapping ToT data for association with itch-suppression
trait_mapping_metadata <- read_tsv("inputs/2024-06-26-top-positive-significant-clusters-orthogroups-annotations.tsv.gz", show_col_types = F) %>%
  mutate(species = gsub("-", " ", species)) %>%
  rename_with(.cols = everything(), function(x){paste0("traitmapping_", x)})

## Use the trait mapping metadata as well as peptigate results and analysis of those results to understand the breakdown of the different orthogroups

* number of proteins in orthogroup
* number of proteins in orthogroup with evidence of expression in salivary gland transcriptomes from the TSA (and percent)
* number of proteins in orthogroup with peptide prediction (and percent)
* number of proteins in orthogroup from ticks (order Ixodida) (and percent)
* number of proteins in orthogroup from chelicerates that suppress itch (and percent)

In [10]:
# number of proteins in orthogroup with peptide prediction (and percent)
orthogroup_peptide_summary <- trait_mapping_metadata %>%
  # join to peptide predictions 
  left_join(peptigate_predictions, by = "traitmapping_locus_tag") %>%
  # join to peptide prediction hits in the tick salivary gland transcriptomes
  left_join(tsa_sg_blastp, by = c("peptigate_peptide_id" = "sgpeptide_blast_qseqid")) %>%
  mutate(peptigate_peptide_prediction = ifelse(is.na(peptigate_peptide_class), "no peptide prediction", "peptide prediction"),
         species_suppresses_itch = ifelse(traitmapping_species %in% evidence_of_itch_suppression_species, "evidence suppresses itch", "no evidence suppresses itch"),
         species_is_a_tick = ifelse(traitmapping_species %in% ticks, "tick", "not tick")) %>%
  group_by(traitmapping_cluster, traitmapping_orthogroup, traitmapping_coefficient, traitmapping_signif_level, traitmapping_signif_fdr) %>%
  summarize(num_proteins_in_orthogroup = n(),
            # number of proteins in orthogroup with peptide prediction (and fraction)
            num_predicted_peptides = sum(peptigate_peptide_prediction == "peptide prediction"),
            fraction_of_orthogroup_with_predicted_peptide = num_predicted_peptides / num_proteins_in_orthogroup,
            # number of protein in orthogroup with signal peptide (and fraction)
            num_predicted_signal_peptides = sum(traitmapping_deepsig_feature == "Signal peptide"),
            fraction_of_orthogroup_with_signal_peptide = num_predicted_signal_peptides / num_proteins_in_orthogroup,
            # number of predicted peptides with predicted signal peptides in orthogroup (and fraction)
            num_predicted_peptides_with_signal_peptide = sum(traitmapping_deepsig_feature == "Signal peptide" & peptigate_peptide_prediction == "peptide prediction"),
            fraction_of_orthogroup_with_predicted_peptide_and_signal_peptide = num_predicted_peptides_with_signal_peptide / num_proteins_in_orthogroup,
            # number of proteins in orthogroup from ticks (order Ixodida) (and fraction), and how many had peptide predictions
            num_tick_proteins_in_orthogroup = sum(species_is_a_tick == "tick"),
            fraction_of_orthogroup_tick_proteins = num_tick_proteins_in_orthogroup / num_proteins_in_orthogroup,
            num_predicted_peptides_from_tick = sum(peptigate_peptide_prediction == "peptide prediction" & species_is_a_tick == "tick"),
            fraction_of_orthogroup_with_predicted_tick_peptides = num_predicted_peptides_from_tick / num_proteins_in_orthogroup,
            # number of proteins in orthogroup from chelicerates that suppress itch (and percent)
            # note itchsuppsp stands for "itch suppression species"
            num_itchsuppsp_proteins_in_orthogroup = sum(species_suppresses_itch == "evidence suppresses itch"),
            fraction_of_orthogroup_itchsuppsp_proteins = num_itchsuppsp_proteins_in_orthogroup / num_proteins_in_orthogroup,
            num_predicted_peptides_from_itchsuppsp = sum(peptigate_peptide_prediction == "peptide prediction" & species_suppresses_itch == "evidence suppresses itch"),
            fraction_of_orthogroup_with_predicted_itchsuppsp_peptides = num_predicted_peptides_from_itchsuppsp / num_proteins_in_orthogroup,
            # calculate whether there is tick & non-tick evidence of itch suppression
            type_of_itch_suppression_evidence = ifelse(num_predicted_peptides_from_itchsuppsp > num_predicted_peptides_from_tick, "chelicerate support", 
                                                       ifelse(num_predicted_peptides_from_tick > 0, "tick support", "no support")),
            # number of proteins (peptides) in orthogroup with evidence of expression in salivary gland transcriptomes from the TSA (and percent)
            num_predicted_peptides_with_sg_blast_hit = sum(!is.na(sgpeptide_blast_bitscore))
           ) %>%
  arrange(type_of_itch_suppression_evidence)

`summarise()` has grouped output by 'traitmapping_cluster', 'traitmapping_orthogroup', 'traitmapping_coefficient', 'traitmapping_signif_level'.
You can override using the `.groups` argument.


In [11]:
orthogroup_peptide_summary_filtered <- orthogroup_peptide_summary %>%
  # filter out orthogroups that had no predicted peptides
  filter(num_predicted_peptides > 0) %>%
  # filter out orthogroups that had no evidence of being expressed in salivary gland
  filter(num_predicted_peptides_with_sg_blast_hit > 0) %>%
  arrange(desc(traitmapping_coefficient))

orthogroup_peptide_summary_filtered 

traitmapping_cluster,traitmapping_orthogroup,traitmapping_coefficient,traitmapping_signif_level,traitmapping_signif_fdr,num_proteins_in_orthogroup,num_predicted_peptides,fraction_of_orthogroup_with_predicted_peptide,num_predicted_signal_peptides,fraction_of_orthogroup_with_signal_peptide,⋯,num_tick_proteins_in_orthogroup,fraction_of_orthogroup_tick_proteins,num_predicted_peptides_from_tick,fraction_of_orthogroup_with_predicted_tick_peptides,num_itchsuppsp_proteins_in_orthogroup,fraction_of_orthogroup_itchsuppsp_proteins,num_predicted_peptides_from_itchsuppsp,fraction_of_orthogroup_with_predicted_itchsuppsp_peptides,type_of_itch_suppression_evidence,num_predicted_peptides_with_sg_blast_hit
<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<int>,<dbl>,⋯,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<chr>,<int>
cluster_40,OG0011284,2.24534051,Yes,No,9,1,0.11111111,0,0.00000000,⋯,8,0.8888889,1,0.11111111,8,0.8888889,1,0.11111111,tick support,1
cluster_40,OG0008888,1.69892817,Yes,No,16,1,0.06250000,7,0.43750000,⋯,16,1.0000000,1,0.06250000,16,1.0000000,1,0.06250000,tick support,1
cluster_40,OG0001774,1.05408448,Yes,No,66,45,0.68181818,43,0.65151515,⋯,51,0.7727273,38,0.57575758,52,0.7878788,39,0.59090909,chelicerate support,34
cluster_33,OG0008102,0.89907992,Yes,No,24,18,0.75000000,17,0.70833333,⋯,24,1.0000000,18,0.75000000,24,1.0000000,18,0.75000000,tick support,13
cluster_27,OG0002194,0.87213413,Yes,No,56,2,0.03571429,1,0.01785714,⋯,45,0.8035714,2,0.03571429,45,0.8035714,2,0.03571429,tick support,1
cluster_27,OG0000189,0.50428119,Yes,No,240,21,0.08750000,22,0.09166667,⋯,193,0.8041667,20,0.08333333,195,0.8125000,20,0.08333333,tick support,15
cluster_46,OG0000746,0.42872398,Yes,No,102,5,0.04901961,2,0.01960784,⋯,71,0.6960784,4,0.03921569,72,0.7058824,4,0.03921569,tick support,2
cluster_27,OG0000194,0.35313516,Yes,No,237,23,0.09704641,NA,NA,⋯,203,0.8565401,21,0.08860759,203,0.8565401,21,0.08860759,tick support,10
cluster_33,OG0000880,0.31400766,Yes,No,109,84,0.77064220,57,0.52293578,⋯,107,0.9816514,82,0.75229358,107,0.9816514,82,0.75229358,tick support,30


In [12]:
write_tsv(orthogroup_peptide_summary_filtered, "outputs/notebooks/20240626_orthogroup_peptide_summary.tsv")

We made a data frame that summarizes our ToT peptigate results.
See a description of the columns below.

* **num_proteins_in_orthogroup**: The number of proteins in the orthogroups assigned by NovelTree.
* Number of proteins in orthogroup with peptide prediction
    * **num_predicted_peptides**: The number of predicted peptides for that orthogroup. For example, if the orthogroup had 10 proteins, 3 may have had peptides predicted from them.
    * **fraction_of_orthogroup_with_predicted_peptide**: Fraction of proteins that had peptides predicted for the orthogroup.
* Number of proteins in orthogroup from ticks (order *Ixodida*), and how many had peptide predictions.
    * **num_tick_proteins_in_orthogroup**: The number of proteins in the orthogroup that were from tick species (order *Ixodida*).
    * **fraction_of_orthogroup_tick_proteins**: The fraction of proteins in the orthogroup that were from tick species (order *Ixodida*).
    * **num_predicted_peptides_from_tick**: The number of peptides that were predicted for the orthogroup that were from a tick species (order *Ixodida*).
    * **fraction_of_orthogroup_with_predicted_tick_peptides**: Fraction of proteins that had peptides predicted from a tick species (order *Ixodida*) for the orthogroup.
* Number of proteins in orthogroup from chelicerates that suppress itch. `itchsuppsp` stands for "itch suppression species"
    * **num_itchsuppsp_proteins_in_orthogroup**: Number of proteins in the orthogroup from itch-suppressing species. This include tick species (order *Ixodida*) as well as two non-tick itch suppressors, *Leptotrombidium deliense* and *Sarcoptes scabiei*.
    * **fraction_of_orthogroup_itchsuppsp_proteins**: Fraction of proteins in the orthogroup from itch-suppressing species.
    * **num_predicted_peptides_from_itchsuppsp**: Number of peptides predicted from itch-suppressing species in the orthogroup.
    * **fraction_of_orthogroup_with_predicted_itchsuppsp_peptides**: Fraction of peptides predicted from itch-suppressiong species in the orthogroup.
* Calculate whether there is tick & non-tick evidence of itch suppression
    * **type_of_itch_suppression_evidence**: Either `chelicerate support`, `tick support`, or `no support`. `Chelicerate support` means that the orthogroup was associated with itch suppression and that there were peptides predicted from both ticks and non-tick itch suppressors that are chelicerates. We think these probably show some of the best evolutionary support for itch suppression but please still think critically about these predictions. `Tick support` means that the orthogroup was associated with itch suppression and peptides were only predicted from tick species. `No support` means that there were no peptides predicted from tick or from the two non-tick itch suppressors.
* Determine whether any member of the orthogroup had a BLAST hit against peptides predicted from the tick salivary gland transcriptomes
    * **num_predicted_peptides_with_sg_blast_hit**: Number of predicted peptides that had hits to peptides predicted from tick salivary gland transcriptomes. We only kept orthogroups that had at least one hit to a tick salivary gland transcriptome peptide. **Note, we need to check the *Amblyomma americanum* hits** -- if these hits are to transcripts that start with the word "Transcript", these hits are from a whole tick transcriptome, not just a salivary gland, so we need to double check that they are expressed in the salivary gland. (The salivary gland transcripts made up over 1/3 of the transcriptome so likelihood is high that they do hit salivary gland transcripts).

In [13]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur ... 10.16

Matrix products: default
BLAS/LAPACK: /Users/taylorreiter/miniconda3/envs/tidyjupyter/lib/libopenblasp-r0.3.26.dylib;  LAPACK version 3.12.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/New_York
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] UpSetR_1.4.0    lubridate_1.9.3 forcats_1.0.0   stringr_1.5.1  
 [5] dplyr_1.1.4     purrr_1.0.2     readr_2.1.5     tidyr_1.3.1    
 [9] tibble_3.2.1    ggplot2_3.5.0   tidyverse_2.0.0

loaded via a namespace (and not attached):
 [1] bit_4.0.5        gtable_0.3.4     jsonlite_1.8.8   compiler_4.3.3  
 [5] crayon_1.5.2     Rcpp_1.0.12      tidyselect_1.2.0 IRdisplay_1.1   
 [9] parallel_4.3.3   gridExtra_2.3    scales_1.3.0     uuid_1.2-0      
[13] fastmap_1.1.1    IRkernel_1.3.2  